In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_2905/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer()
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
20,model_5_9_7,0.871119,0.679789,0.871602,0.808338,0.024111,0.045027,0.029258,0.037607,"Hidden Size=[30, 15], regularizer=0.02, learni..."
22,model_5_9_6,0.870063,0.690825,0.874030,0.813851,0.024308,0.043475,0.028705,0.036525,"Hidden Size=[30, 15], regularizer=0.02, learni..."
25,model_5_9_5,0.868833,0.701861,0.876145,0.819194,0.024538,0.041923,0.028223,0.035476,"Hidden Size=[30, 15], regularizer=0.02, learni..."
27,model_5_9_4,0.867425,0.712853,0.877955,0.824354,0.024802,0.040378,0.027811,0.034464,"Hidden Size=[30, 15], regularizer=0.02, learni..."
29,model_5_9_3,0.865817,0.723720,0.879441,0.829290,0.025103,0.038850,0.027472,0.033496,"Hidden Size=[30, 15], regularizer=0.02, learni..."
30,model_8_6_0,0.865809,0.940147,0.871295,0.934362,0.025104,0.014604,0.015194,0.014882,"Hidden Size=[30, 15], regularizer=0.1, learnin..."
32,model_5_9_2,0.863983,0.734346,0.880561,0.833933,0.025446,0.037356,0.027217,0.032584,"Hidden Size=[30, 15], regularizer=0.02, learni..."
34,model_5_9_1,0.861906,0.744643,0.881279,0.838232,0.025834,0.035908,0.027053,0.031741,"Hidden Size=[30, 15], regularizer=0.02, learni..."
45,model_5_9_0,0.859556,0.754502,0.881560,0.842126,0.026274,0.034521,0.026989,0.030977,"Hidden Size=[30, 15], regularizer=0.02, learni..."
54,model_9_9_12,0.848663,0.669859,0.828227,0.756101,0.028312,0.055633,0.022768,0.040167,"Hidden Size=[45, 20], regularizer=0.02, learni..."
